In [1]:
import torch
import torch.nn as nn
import random
import numpy as np
import pandas as pd
import json
import cv2
import os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from functools import partial
from dataclasses import dataclass
from collections import OrderedDict
import torchvision
from IPython.display import clear_output
import matplotlib.pyplot as plt
import tqdm
from PIL import ImageFilter, Image
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import math
#import torch.utils.model_zoo as model_zoo
from torch.nn import init

from network.meso import Meso4, MesoInception4
from network.models import model_selection, return_pytorch04_xception
from network.lstm_deepfake import LSTMDF
 
from pipeline.metrics import accuracy, log_loss, accuracy_b, log_loss_b, accuracy_b_mean
from pipeline.model_methods import validate_img, train_img, validate_vid, train_vid
from pipeline.data_loaders import load_img_dataset
from pipeline.image_extracting import extract_faces, FastMTCNN, InceptionResnetV1, extract_faces_dlib, MTCNN

from pipeline.blazeface import BlazeFace


random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True
PATH = "model.h5"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
d:\users\user\appdata\local\programs\python\python36\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
facedet = BlazeFace().to(device)
facedet.load_weights("/pipeline/blazeface.pth")
facedet.load_anchors("/pipeline/anchors.npy")
_ = facedet.train(False)

from pipeline.helpers.read_video_1 import VideoReader
from pipeline.helpers.face_extract_1 import FaceExtractor

frames_per_video = 17

video_reader = VideoReader()
video_read_fn = lambda x: video_reader.read_frames(x, num_frames=frames_per_video)
face_extractor = FaceExtractor(video_read_fn, facedet)

In [2]:
y_train = pd.read_csv('data\\metadata.csv')
len(y_train[y_train.label == 1]), len(y_train[y_train.label == 0])

(71210, 13129)

In [3]:
x_train = r'D:\Machine Learning\deepfake-detection\data\train_set'
x_val = r'D:\Machine Learning\deepfake-detection\data\validation_set'
x_test = r'D:\Machine Learning\deepfake-detection\data\cross_val'

In [4]:
model, *_ = model_selection(modelname='xception', num_out_classes=2, dropout=None)
#model = LSTMDF(dropout=0.5)
#model = models.resnet18(pretrained=True)

In [5]:
%%script false

for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Sequential(nn.Linear(512, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2))

Couldn't find program: 'false'


In [6]:
%%script false
model.load_state_dict(torch.load('pretrained/best.pkl'))

if isinstance(model, torch.nn.DataParallel):
    model = model.module

model.eval()

Couldn't find program: 'false'


In [7]:
#%%script false

model.load_state_dict(torch.load("pretrained/xception/full_raw.pth"))
model.eval()

TransferModel(
  (model): Xception(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (block1): Block(
      (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (rep): Sequential(
        (0): SeparableConv2d(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
          (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        )
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace)
    

In [8]:
#%%script false

exceptions = ['last_linear']

for name, param in model.named_parameters():
    for e in exceptions:
        if e in name:
            param.requires_grad = True
        else:
            param.requires_grad = False

In [9]:
loss =  torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) 
optimizer = torch.optim.Adam(model.model.last_linear.parameters(), lr=1e-3) 
#optimizer = torch.optim.SGD(model.model.last_linear.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [10]:
model = model.to(device)

In [11]:
fast_mtcnn = FastMTCNN(
    stride=4,
    resize=1,
    margin=14,
    factor=0.9,
    #min_face_size=60,
    #keep_all=True,
    select_largest=False,
    device=device
)
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((333, 333)),
    torchvision.transforms.CenterCrop((299, 299)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))
    #torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
#resnet = InceptionResnetV1(pretrained='vggface2', device=device).eval()

In [12]:
#%%script false

'''
all_c40

noisy 224 4 1 14 0.6 false false 10 20 0.99      | Validation: metrics  0.61 loss  1.5346809851754295
noisy 244 4 1 14 0.6 false false 10 20 0.99      | Validation: metrics  0.55 loss  1.805389928239515
noisy 256 4 1 14 0.6 false false 10 20 0.99      | Validation: metrics  0.625 loss  1.6223815120170137 | Test: 0.68081 LL 0.5812 ACC
noisy 299 4 1 14 0.6 false false 10 20 0.99      | Validation: metrics  0.59 loss  1.5987016558059581
'''

'''
full_raw dlib mean

noisy 224 4 1 14 0.6 false false 10 20 0.99      | 
noisy 244 4 1 14 0.6 false false 10 20 0.99      | 
noisy 256 4 1 14 0.6 false false 10 20 0.99      | 
noisy 299 4 1 14 0.6 false false 10 20 0.99      | 
noisy 333 4 1 14 0.9 false false 10 20 0.99      | Validation: metrics  0.635 loss  0.6700506503431825
'''

validate_vid(model, x_test, y_train, loss, accuracy_b_mean, device, 10, fast_mtcnn, transforms,
             print_results=True, 
             inference=nn.Softmax(dim=1), delimeter=20, remove_noise=False)

0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
0 tensor(0, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
0 tensor(1, device='cuda:0')
1 tensor(1, device='cuda:0')
1 tensor(0, de

0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
0 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, device='cuda:0')
1 tensor(0, de

(0.635, 0.6700506503431825)

In [13]:
#faces = torch.FloatTensor(extract_faces(r'D:\Machine Learning\deepfake-detection\data\train_set\aabqyygbaa.mp4', fast_mtcnn, transforms, limit=1, delimeter=1))

In [14]:
#plt.imshow(faces[0].permute(1,2,0))
#plt.show()

In [15]:
metric, lost = train_v1id(model, loss, optimizer, scheduler, x_train, x_test, y_train, accuracy_b, device, fast_mtcnn, 
                         transforms, 
                         epochs=3, 
                         batch_size=5, 
                         del_net=False, 
                         useInference=False,
                         inference=nn.Softmax(dim=1), 
                         useScheduler=True,
                         checkpoint=0.6,
                         limit=1,
                         delimeter=25,
                         remove_noise=False
                      )

NameError: name 'train_v1id' is not defined

In [ ]:
from operator import itemgetter
min_lost = min(enumerate(lost), key=itemgetter(1))[0] 
min_metrics = max(enumerate(metric), key=itemgetter(1))[0] 
print("Loss optim:", lost[min_lost], min_lost + 1)
print("Metrics optim:", metric[min_metrics], min_metrics + 1)

In [ ]:
plt.plot(metric, label='model')
plt.legend()
plt.title('Validation metrics')

In [ ]:
plt.plot(lost, label='model')
plt.legend()
plt.title('Validation Loss')

In [ ]:
validate_vi1d(model, x_val, y_train, loss, accuracy_b, device, 30, fast_mtcnn, transforms, 
             show_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, print_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, show_results=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
validate_vid(model, x_val, y_train, loss, log_loss, device, 30, fast_mtcnn, transforms, show_graphic=True, inference=nn.Softmax(dim=1), delimeter=5)

In [ ]:
#torch.save(model.state_dict(), '0.657.pth')